# 1. Import Dependencies

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 2. Load Model

In [5]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

# 3. Make Detections

In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    #resize
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 256,256)
    input_img = tf.cast(img, dtype = tf.int32)

    #detection
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    print(keypoints_with_scores)

    cv2.imshow('frame', frame)

    if cv2.waitKey(10) & 0xFF==ord('q') :
        break

cap.release()
cv2.destroyAllWindows()

# 4. Draw Keypoints

In [44]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)


# 5. Draw Edges

In [45]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [46]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

# 6. Real Time Detection

In [47]:
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:# 6 kişiyide alır
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [50]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    #resize
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 192,256)
    input_img = tf.cast(img, dtype = tf.int32)

    #detection
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    #draw
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)

    cv2.imshow('frame', frame)

    if cv2.waitKey(10) & 0xFF==ord('q') :
        break

cap.release()
cv2.destroyAllWindows()

# 7. Video Detection 

In [57]:
cap = cv2.VideoCapture('novak.mp4')

while cap.isOpened():
    ret, frame = cap.read()

    #resize
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 352,640)#32x
    input_img = tf.cast(img, dtype = tf.int32)

    #detection
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    #draw
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.15)

    cv2.imshow('frame', frame)

    if cv2.waitKey(10) & 0xFF==ord('q') :
        break

cap.release()
cv2.destroyAllWindows()